In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import io
import re
import osmnx as ox
import geopandas
import json
import cartopy.crs as ccrs
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from datetime import datetime, date
from dateutil import parser
import geopy
from geopy.geocoders import Nominatim, GoogleV3
from math import radians, cos, sin, asin, sqrt
from collections import Counter
from itertools import groupby
from operator import itemgetter
import itertools
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from sklearn.preprocessing import StandardScaler
#import folium

In [ ]:
xls = pd.ExcelFile(os.path.expanduser("")) 
df3 = pd.read_excel(xls, 'VehicleSurvey')
df4 = pd.read_excel(xls, 'ActivitySurvey')
df4['D_1'] = df4['D_1'].fillna(value = '0')
#df4.head()

In [ ]:
'''running this cell takes some time'''
df6 = pd.read_csv(os.path.expanduser(""),
                  names=['timestamp','long','diffseconds','instantspeed','diffdistance','accuracy',
                         'lati','device_id'])

def TimeParse(x):
    a = ''.join([re.split(':|}|{|', x)[4], ':', re.split(':|}|{|', x)[5], 
            ':', re.split(':|}|{|', x)[6]]).strip('"')
    return parser.parse(a)

df6['time'] = df6['timestamp'].apply(lambda x: TimeParse(x))
df6['ID'] = df6['device_id'].apply(lambda x: re.split(':|}|{|',x)[3].strip('"'))
df6['lon'] = df6['long'].apply(lambda x: re.split(':|}|{|',x)[3].strip('"')).astype(float)
df6['lat'] = df6['lati'].apply(lambda x: re.split(':|}|{|',x)[3].strip('"')).astype(float)
df6['speed'] = df6['instantspeed'].apply(lambda x: re.split(':|}|{|',x)[3].strip('"')).astype(float)
df6['diff_distance'] = df6['diffdistance'].apply(lambda x: re.split(':|}|{|',x)[3].strip('"')).astype(float)
df6['diff_time'] = df6['diffseconds'].apply(lambda x: re.split(':|}|{|',x)[3].strip('"')).astype(float)
df6['day'] = df6['time'].apply(lambda x: x.date())

df6.head()

In [ ]:
'''df8 is all stops of vehicles that have been tasked for household delivery'''

df8 = df4.merge(df6, left_on=['Survey ID','DS_4','DS_5'], right_on=['ID','lon','lat'], how='inner')
df5 = df4[(df4['D_1'].isin(['2','5'])) & (df4['D_2'] == 8.0)]
df8 = df8[df8['Survey ID'].isin(df5['Survey ID'].unique())]
#df8 = pd.concat([df8.iloc[:,2:12],df8.iloc[:,16:23],df8.iloc[:,34:36]],axis=1)
df8 = df8.drop_duplicates(['DS_3'])
df8 = df8.drop_duplicates(['Survey ID','time'])
df8 = df8.reset_index(drop=False)
df8['date'] = df8['time'].apply(lambda x: x.date())
df8['hour'] = df8['time'].apply(lambda x: x.hour)

stopkey1_df8 = df8[df8['D_1'] == '1']['DS_3'].unique()
array_stop1_df8 = np.array([[re.split('-',key)[0],re.split('-',key)[1]] for key in stopkey1_df8])
df_stop1 = pd.DataFrame(data = array_stop1_df8,
                        columns = ['ID','pick_up'])
code_stop1_df8 = [df8[df8['DS_3'] == key]['D_2'].unique() for key in stopkey1_df8]
df_stop1['pick_up_code'] = code_stop1_df8

stopkey2_df8 = df8[df8['D_1'] == '2']['DS_3'].unique()
array_stop2_df8 = np.array([[re.split('-',key)[0],re.split('-',key)[1]] for key in stopkey2_df8])
df_stop2 = pd.DataFrame(data = array_stop2_df8,
                        columns = ['ID','deliver'])
code_stop2_df8 = [df8[df8['DS_3'] == key]['D_2'].unique() for key in stopkey2_df8]
df_stop2['deliver_code'] = code_stop2_df8

df_stop_both = df_stop1.merge(df_stop2, on = 'ID', how = 'inner')
df_stop_both['pick_up'] = df_stop_both['pick_up'].astype(int)
df_stop_both['deliver'] = df_stop_both['deliver'].astype(int)
df_stop_both = df_stop_both[df_stop_both['pick_up'] < df_stop_both['deliver']]

## condition for filter redundant rows
d = {'pick_up':'max'}

#'''df13 finds the trip(pick up and deliver stops) for all vehicles'''
df13 = df_stop_both.groupby(['ID','deliver']).aggregate(d)
df13.index.names = ['level_1', 'level_2']
df13['ID'] = df13.index.get_level_values('level_1')
df13['deliver'] = df13.index.get_level_values('level_2')

#'''df14 finds the number of delivery stops for each vehicle after it's each picking up stop'''
df14 = pd.DataFrame(df13.groupby(['ID','pick_up']).size().rename('count'))

#'''df15 finds the frequency(distribution) of the number of delivery stops'''
#'''473 delivery stops? not all are delivery to household'''
df15 = pd.DataFrame(df14.groupby(['count']).size().rename('freq'))
df15['share'] = df15['freq'] / df15['freq'].sum()

#'''df41 finds all tours with establishments of pick up and deliver'''
df41 = df13.merge(df_stop_both, on = ['ID','pick_up','deliver'], how = 'inner')
df41 = df41.sort_values(by = ['ID','pick_up','deliver'])
df41['pick_up_ID'] = df41['ID'] + '-' + df41['pick_up'].astype(str)
df41['deliver_ID'] = df41['ID'] + '-' + df41['deliver'].astype(str)

df41['pick_up_code'] = df41['pick_up_code'].astype(str)
df41['pick_up_code'] = df41['pick_up_code'].apply(lambda x: x.strip('[]'))
df41['deliver_code'] = df41['deliver_code'].astype(str)
df41['deliver_code'] = df41['deliver_code'].apply(lambda x: x.strip('[]'))

#df_stop_both.head()
df41.head()

In [ ]:
'''df43 records delivery tours(without pick up for now)'''

#pd.DataFrame([np.array(df41[df41['pick_up_ID'] == key]['deliver_code'].values, dtype = object) 
              #for key in df41['pick_up_ID'].unique()])
#len(np.array(df41[df41['pick_up_ID'] == df41['pick_up_ID'].unique()[0]]['deliver_code'].values, dtype = object))
df43 = pd.DataFrame([df41[df41['pick_up_ID'] == key]['deliver_code'].values for key in df41['pick_up_ID'].unique()])
df43 = df43.T
df43 = df43.fillna(0)
keep_index = [ind for ind in df43.columns if ((df43[ind] == '8.').any())]
#df43 = df43.T
df43 = df43.iloc[:,keep_index]
#df43 = df43.T

pd.DataFrame(pd.DataFrame(data = [sum(~df43[ind][:-1].isin(['8.', 0])) for ind in df43.columns],
            columns = ['number of non-house stops']).groupby(['number of non-house stops']).size().rename('count'))
#df43[0].bfill().ffill().is_unique
#[col for col in df.columns if not df[col].is_unique]

In [ ]:
'''to plot on qgis with tour id'''
all_house_index = [ind for ind in df43.columns if ((df43[ind][:-1].isin(['8.',0]).all()))]

all_house_stop_record = df8[df8['DS_3'].isin(all_house_stop_ID)]

'''all_house_stop_ID find stop_ID(p/d) for all-house tours'''
all_house_stop_ID = np.hstack([df41[df41['pick_up_ID'].
                                       isin(df41['pick_up_ID'].unique()[all_house_index])]['deliver_ID'].values , 
                                  df41['pick_up_ID'].unique()[all_house_index]])

all_house_pickup_stop_ID = df41['pick_up_ID'].unique()[all_house_index]
all_house_delivery_stop_ID = df41[df41['pick_up_ID'].
                                       isin(df41['pick_up_ID'].unique()[all_house_index])]['deliver_ID'].values

all_house_pickup_stop_record = df8[df8['DS_3'].isin(all_house_pickup_stop_ID)]
all_house_pickup_stop_record = all_house_pickup_stop_record.reset_index(drop=True)
all_house_pickup_stop_record = all_house_pickup_stop_record.merge(df64, on = 'index', how = 'inner')
#all_house_pickup_stop_record.to_csv('all_house_pickup_stop.csv')
all_house_delivery_stop_record = df8[df8['DS_3'].isin(all_house_delivery_stop_ID)]
all_house_delivery_stop_record = all_house_delivery_stop_record.reset_index(drop=True)
all_house_delivery_stop_record = all_house_delivery_stop_record.merge(df64, on = 'index', how = 'inner')
#all_house_delivery_stop_record.to_csv('all_house_delivery_stop.csv')
all_house_pickup_stop_record

In [ ]:
def partition(values, indices):
    idx = 0
    for index in indices:
        sublist = []
        while idx < len(values) and values[idx] < index:
            sublist.append(values[idx])
            idx += 1
        if sublist:
            yield np.hstack([sublist, index])
        else:
            yield [index]

def check_tour(sequence):
    #if ('1' or '2') not in sequence: return None
    if (('1' not in sequence) and ('2' not in sequence)): return None
    if (('1' in sequence) and ('2' not in sequence)): return 1
    if (('2' in sequence) and ('1' not in sequence)): return 2
    if ('1' and '2') in sequence:
        #for s in sequence:
        ind1 = [i for i,s in enumerate(sequence) if s == '1']     # index of pickup in the tour sequence
        ind2 = [i for i,s in enumerate(sequence) if s == '2']
        #tour_ind = list(get_tour(ind1, ind2))
        tour_ind = list(partition(ind1, ind2))     # index in separate tours
        tour = [[sequence[tour_ind[j][i]] for i in range(len(tour_ind[j]))] 
                for j in range(len(tour_ind))]     # separate tours
        return tour            
            
def refine_tour(tour):
    if tour == None: return None
    if tour == 1: return 1
    if tour == 2: return 2
    else:
        for i in range(len(tour)):
            if tour[i] == [u'2'] and i>0:
                for j in reversed(range(i)):
                    if tour[j] != [u'2']:
                        tour[j].append(u'2')
                        break
        
        alltour = []
        for subtour in tour:
            if subtour!= [u'2']:
                alltour.append(subtour)
        #for i in range(len(tour)):
            #if tour[i] == [u'2']:
                #tour[i] = [0]
        return alltour
        
def get_tour_type(tour):
    if tour:
        times_pickup = len([s for s in tour if s == '1'])
        times_deliver = len([s for s in tour if s == '2'])
        if times_pickup == 1 and times_deliver == 1:
            return 'direct tour'
        elif times_pickup == 1 and times_deliver > 1:
            return 'unloading'
        elif times_pickup > 1 and times_deliver == 1:
            return 'loading'
        elif times_pickup > 1 and times_deliver > 1:
            return 'mixed'
        else:
            return 'NA'
        
def get_num_deliver_stop(tour):
    if tour:
        return len([s for s in tour if s == '2'])
    
def get_num_pickup_stop(tour):
    if tour:
        return len([s for s in tour if s == '1'])

In [ ]:
def get_tour_index(df):
    if df[5] not in [1,2,None]:
        tour_length = [len(s) for s in df[5]]
        sublist = []
        for i in range(len(tour_length)):
            front_ind = sum(tour_length[:i+1]) - tour_length[0]
            end_ind = sum(tour_length[:i+1])
            sublist.append(df[4][front_ind:end_ind])
            
        return sublist
    
def get_tot_dist(df):
    #return sum([df8[df8['index'] == ind]['diff_distance'].values for ind in x if ind in list(df8['index'])])[0] 
    x = df['tour_index_number']
    y = df['ID']
    if x == None: return None
    else:
        if len(x) == 1:
            f,l = x[0][0], x[0][-1]
            first = df6[df6['time'] == df8[df8['index'].isin(range(f,l+1))].iloc[0]['time']].index.values[0]
            last = df6[df6['time'] == df8[df8['index'].isin(range(f,l+1))].iloc[-1]['time']].index.values[0]
            return sum(df6[(df6['ID'] == y) & (df6.index.isin(range(first,last+1)))]['diff_distance'])/1000.
        elif len(x) > 1:
            record = []
            for i in range(len(x)):
                f,l = x[i][0], x[i][-1]
                first = df6[df6['time'] == df8[df8['index'].isin(range(f,l+1))].iloc[0]['time']].index.values[0]
                last = df6[df6['time'] == df8[df8['index'].isin(range(f,l+1))].iloc[-1]['time']].index.values[0]
                record.append(sum(df6[(df6['ID'] == y) & (df6.index.isin(range(first,last+1)))]['diff_distance'])/1000.)
            return record
        else: return None

def get_tot_time(df):
    #return sum([df8[df8['index'] == ind]['diff_distance'].values for ind in x if ind in list(df8['index'])])[0] 
    x = df['tour_index_number']
    y = df['ID']
    if x == None: return None
    else:
        if len(x) == 1:
            f,l = x[0][0], x[0][-1]
            first = df6[df6['time'] == df8[df8['index'].isin(range(f,l+1))].iloc[0]['time']].index.values[0]
            last = df6[df6['time'] == df8[df8['index'].isin(range(f,l+1))].iloc[-1]['time']].index.values[0]
            return sum(df6[(df6['ID'] == y) & (df6.index.isin(range(first,last+1)))]['diff_time'])/60.
        elif len(x) > 1:
            record = []
            for i in range(len(x)):
                f,l = x[i][0], x[i][-1]
                first = df6[df6['time'] == df8[df8['index'].isin(range(f,l+1))].iloc[0]['time']].index.values[0]
                last = df6[df6['time'] == df8[df8['index'].isin(range(f,l+1))].iloc[-1]['time']].index.values[0]
                record.append(sum(df6[(df6['ID'] == y) & (df6.index.isin(range(first,last+1)))]['diff_time'])/60.)
            return record
        else: return None
        
def get_tour_dist(df):
    record = []
    for i in range(len(df)):
        item = df.iloc[i]['total_distance']
        if item != None:
            if type(item) == list:
                for em in item:
                    record.append(em)
            elif type(item) == float:
                record.append(item)
                
    return record

def get_tour_time(df):
    record = []
    for i in range(len(df)):
        item = df.iloc[i]['total_time']
        if item != None:
            if type(item) == list:
                for em in item:
                    record.append(em)
            elif type(item) == float:
                record.append(item)
                
    return record


In [ ]:
def get_tour_process(df):
    df58 = pd.DataFrame(df.groupby(['date','Survey ID'])['D_1'].apply(list).rename('stop_sequence'))  # cool stuff
    df59 = pd.DataFrame(df.groupby(['date','Survey ID'])['time'].apply(list).rename('time'))
    df60 = pd.DataFrame(df.groupby(['date','Survey ID'])['D_2'].apply(list).rename('establishment'))
    df61 = pd.DataFrame(df.groupby(['date','Survey ID'])['D_4'].apply(list).rename('type_product'))
    df62 = pd.DataFrame(df.groupby(['date','Survey ID']).index.apply(list).rename('index_number'))
    df58 = pd.concat([df58,df59,df60,df61,df62], axis = 1)
    df58['tour'] = df58['stop_sequence'].apply(lambda x: check_tour(x))
    df58['tour'] = df58['tour'].apply(lambda x: refine_tour(x))
    alltour = []
    for tour in df58[~df58['tour'].isin([None,1,2])]['tour']:
        for subtour in tour:
            if subtour != [0]:
                alltour.append(subtour)
                
    values, counts = np.unique(alltour, return_counts=True)
    df59 = pd.DataFrame(data = values, columns = ['tour'])
    df59['counts'] = counts
    
    alltour_type = [get_tour_type(tour) for tour in alltour]
    val, coun = np.unique(alltour_type, return_counts=True)
    df60 = pd.DataFrame(data = val, columns = ['tour_type'])
    df60['counts'] = coun
    
    return df58, df59, df60

In [ ]:
#set(chain.from_iterable(dict_stop1))
'''def get_tour_index(df):
    if df[5] != 2:
        tour_length = [len(s) for s in df[5]]
        sublist = []
        for i in range(len(tour_length)):
            front_ind = sum(tour_length[:i+1]) - tour_length[0]
            end_ind = sum(tour_length[:i+1])
            sublist.append(df[4][front_ind:end_ind])
            
        return sublist'''

df63 = get_tour_process(all_house_stop_record)[0]
df63['tour_index_number'] = df63.apply(get_tour_index, axis = 1)    # cool stuff
#df63['total_distance'] = df63['tour_index_number'].apply(lambda x: get_tot_dist(x))
df63['ID'] = np.zeros(len(df63))
for i in range(len(df63)):
    df63['ID'].iloc[i] = df63.index[i][1]
df63['total_distance'] = df63.apply(get_tot_dist, axis = 1)
df63['total_time'] = df63.apply(get_tot_time, axis = 1)
#df63['is_overnight'] = df63.apply(identify_overnight_tour, axis = 1)
#df63

In [ ]:
'''df8 is all stops of vehicles that have been tasked for household delivery'''

def get_tour_step1(estab,df4,df6):
    df_estab = df4[(df4['D_1'].isin(['2','5'])) & (df4['D_2'] == estab)]
    #df_estab = pd.concat([df_estab.iloc[:,2:12],df_estab.iloc[:,16:23]],axis=1)
    df = df4.merge(df6, left_on=['Survey ID','DS_4','DS_5'], right_on=['ID','lon','lat'], how='inner')
    df = df[df['Survey ID'].isin(df_estab['Survey ID'].unique())]
    df = pd.concat([df.iloc[:,2:12],df.iloc[:,16:23],df.iloc[:,34:36]],axis=1)
    df = df.drop_duplicates(['DS_3'])
    df = df.drop_duplicates(['Survey ID','time'])
    df = df.reset_index(drop=False)
    
    stopkey1 = df[df['D_1'] == '1']['DS_3'].unique()
    array_stop1 = np.array([[re.split('-',key)[0],re.split('-',key)[1]] for key in stopkey1])
    df_stop1 = pd.DataFrame(data = array_stop1,
                        columns = ['ID','pick_up'])
    df_stop1['pick_up'] = pd.to_numeric(df_stop1['pick_up'], errors='coerce')  # deal with NAN values
    df_stop1['pick_up'] = df_stop1['pick_up'].fillna(0)
    code_stop1 = [df[df['DS_3'] == key]['D_2'].unique() for key in stopkey1]
    df_stop1['pick_up_code'] = code_stop1
    
    stopkey2 = df[df['D_1'] == '2']['DS_3'].unique()
    array_stop2 = np.array([[re.split('-',key)[0],re.split('-',key)[1]] for key in stopkey2])
    df_stop2 = pd.DataFrame(data = array_stop2,
                        columns = ['ID','deliver'])
    df_stop2['deliver'] = pd.to_numeric(df_stop2['deliver'], errors='coerce')  # deal with NAN values
    df_stop2['deliver'] = df_stop2['deliver'].fillna(0)
    code_stop2 = [df[df['DS_3'] == key]['D_2'].unique() for key in stopkey2]
    df_stop2['deliver_code'] = code_stop2
    
    df_stop_both = df_stop1.merge(df_stop2, on = 'ID', how = 'inner')
    df_stop_both['pick_up'] = df_stop_both['pick_up'].astype(int)
    #df_stop_both['deliver'] = pd.to_numeric(df_stop_both['deliver'], errors='coerce')  # deal with NAN values
    #df_stop_both['deliver'] = df_stop_both['deliver'].fillna(0)
    df_stop_both['deliver'] = df_stop_both['deliver'].astype(int)
    df_stop_both = df_stop_both[df_stop_both['pick_up'] < df_stop_both['deliver']]

    ## condition for filter redundant rows
    d = {'pick_up':'max'}

    '''finds the trip(pick up and deliver stops) for all vehicles'''
    df_1 = df_stop_both.groupby(['ID','deliver']).aggregate(d)

    '''finds the number of delivery stops for each vehicle after it's each picking up stop'''
    df_2 = pd.DataFrame(df_1.groupby(['ID','pick_up']).size().rename('count'))
    
    df_3 = pd.DataFrame(df14.groupby(['count']).size().rename('freq'))
    df_3['share'] = df_3['freq'] / df_3['freq'].sum()
    
    df = df_1.merge(df_stop_both, on = ['ID','pick_up','deliver'], how = 'inner')
    df = df.sort_values(by = ['ID','pick_up','deliver'])
    df['pick_up_ID'] = df['ID'] + '-' + df['pick_up'].astype(str)
    df['pick_up_code'] = df['pick_up_code'].astype(str)
    df['pick_up_code'] = df['pick_up_code'].apply(lambda x: x.strip('[]'))
    df['deliver_code'] = df['deliver_code'].astype(str)
    df['deliver_code'] = df['deliver_code'].apply(lambda x: x.strip('[]'))
    
    return df


In [ ]:
plt.bar(x = df15.index, height = df15['share'], width = 0.45, color = 'r', align = 'center', 
        label = 'household delivery vehicle')
plt.bar(x = df18.index + 0.45, height = df18['share'], width = 0.45, color = 'b', align = 'center', 
        label = 'all vehicles')
plt.xlabel('number of delivery stops')
plt.ylabel('frequency')
plt.legend()
plt.title('distribution of number of delivery stops')

In [ ]:
'''distribution of delivery origins (pickup locations)'''

ind_D2 = ['company office/headquarters', 'WH/DC',
          'transfer terminal/ports', 'manufacturer/factory',
          'construction site', 'farm',
          'retail/food establishment', 'private residence',
          'other']

df22 = pd.DataFrame(df20.groupby('D_2').size().rename('freq'))

ind_D2_house = ['company office/headquarters', 'WH/DC',
          'manufacturer/factory',
        'farm',
          'retail/food establishment', 'private residence',
          'other']

df21 = pd.DataFrame(df19.groupby('D_2').size().rename('freq'))


plt.subplots(3,1, figsize = (12,10))
'''same horizontal axis?'''

plt.subplot(311)
plt.barh(y = ind_D2_house, width = df21.freq)
plt.title('distribution of pickup locations for vehicles deliver to household')

plt.subplot(312)
plt.barh(y = ind_D2, width = df22.freq)
plt.title('distribution of pickup locations for all vehicles')

plt.subplot(313)
plt.barh(y = ind_D2, width = df22.freq, label = 'all vehicles')
plt.barh(y = ind_D2_house, width = df21.freq, label = 'vehicles to household')
ind22 = np.arange(len(df22))
ind21 = np.arange(len(df21))
height = 0.4
#plt.barh(y = ind_D2, width = df22.freq, height = height, label = 'all')
#plt.barh(y = ind_D2_house + height, width = df21.freq, height = height, label = 'household')
plt.legend()
plt.title('distribution of pickup locations')
plt.savefig('pickup_origin.png')
#plt.xticks(rotation='vertical')
#plt.xticks(rotation=65)

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [ ]:
'''analyze repetitiveness bahavior?'''
'''more complete list of malls (over 200 malls)'''
malls = pd.read_csv(os.path.expanduser("~/Documents/malls.csv"))
def process_malls(mall_location,distance):
    mall_location['possible_match'] = range(len(mall_location))
    for i in range(len(mall_location)):
        lon1, lat1 = mall_location['lon'].iloc[i], mall_location['lat'].iloc[i]
        df_distance_try = pd.DataFrame(data = [haversine(lon1, lat1, df_retail['DS_4'].iloc[ind], df_retail['DS_5'].iloc[ind]) 
                                               for ind in range(len(df_retail))],
                                       columns = ['distance'])
        mall_location['possible_match'].iloc[i] = np.array(df_distance_try[df_distance_try['distance'] < distance].index.values,
                                                           dtype = object)
    
    mall_location['possible_match_len'] = mall_location['possible_match'].apply(lambda x: len(x))
    return mall_location
    
df65 = process_malls(malls,0.3)


In [ ]:
'''look into location of malls'''
'''running this cell takes a while'''
'''mall is at 313 Orchard Road, find 176, 181, 290, 391 all working, 
or some other close location, is this correct?'''

mall_location = pd.read_excel(os.path.expanduser("~/Documents/List of malls in SG.xlsx"))
mall_location['area'] = mall_location.index
mall_location = mall_location.reset_index(drop=True)

mall_lonlat = [[mall_location['Latitude'].iloc[i], mall_location['Longitude'].iloc[i]] 
               for i in mall_location.index]

#mall_lonlat = [[format(mall_location['Longitude'].iloc[i], '.5f'), format(mall_location['Latitude'].iloc[i],'.5f')] 
               #for i in mall_location.index]
mall_location['latlon'] = mall_lonlat
mall_location['latlon'] = mall_location['latlon'].astype(str)
mall_location['latlon'] = mall_location['latlon'].apply(lambda x: x.strip('[]'))

mall_location['lon'] = mall_location['Longitude'].apply(lambda x: round(x,5))
mall_location['lat'] = mall_location['Latitude'].apply(lambda x: round(x,5))

df_retail = df9[(df9['D_1'].isin(['2','5'])) & (df9['D_2'] == 7.0) ]
df_retail = df_retail.drop_duplicates(['DS_3'])
df_retail = df_retail.drop_duplicates(['Survey ID','time'])
df_retail = df_retail.reset_index(drop=False)

mall_location['possible_match'] = range(len(mall_location))
for i in range(len(mall_location)):
    lon1, lat1 = mall_location['Longitude'].iloc[i], mall_location['Latitude'].iloc[i]
    df_distance_try = pd.DataFrame(data = [haversine(lon1, lat1, df_retail['DS_4'].iloc[ind], df_retail['DS_5'].iloc[ind]) 
                                           for ind in range(len(df_retail))],
                                   columns = ['distance'])
    mall_location['possible_match'].iloc[i] = np.array(df_distance_try[df_distance_try['distance'] < 0.5].index.values,
                                                   dtype = object)
    
mall_location['possible_match_len'] = mall_location['possible_match'].apply(lambda x: len(x))

'''possible service timeout issue'''    
'''def get_location(x):
    geolocator = Nominatim()
    #geolocator = GoogleV3()
    #location = geolocator.geocode(x)
    location = geolocator.reverse(x)
    return location.address

mall_location['location'] = mall_location['lonlat'].apply(lambda x : get_location(x))'''

mall_location.head()

In [ ]:
def get_location(x):
    geolocator = Nominatim()
    #geolocator = GoogleV3()
    #location = geolocator.geocode(x)
    location = geolocator.reverse(x)
    return location.address

mall_location['location'] = mall_location['lonlat'].apply(lambda x : get_location(x))

mall_location.head()

In [ ]:
#df_retail[103:120]
#mall_location
'''0,7,11,13,25,36,59,73,90,95'''
typical_malls = ['313@Somerset','City Square Mall',
                 'ION Orchard','Marina Bay Sands',
                 'Suntec City','Tampines Mall',
                 'Junction 8', 'Hougang Mall',
                 'Vivocity','JEM']

'''find all index that is possibly deliver to malls'''
list(set(np.hstack(mall_location['possible_match'])))

In [ ]:
ind_D7_mall = ['25x25x25cm', '100x100x100cm', '200x100x100cm', 'one pallet',
               'multiple pallet','1/4 truck', '1/2 truck', 'full truck']

df46 = pd.DataFrame(df_retail[df_retail.index.isin(list(set(np.hstack(mall_location['possible_match']))))].
             groupby(['D_7']).size().rename('counts'))
plt.barh(y = ind_D7_mall, width = df46.counts)
for i, v in enumerate(df46.counts):
    plt.text(v, i, " "+str(v), color='black', va='center', fontweight='bold')
plt.title('distribution of cargo volumn for vehicles that possibly deliver to mall')

In [ ]:
'''for a deliver trip(between 2 stops at picking up points), 
   most have 0 or 1 deliver stop at private residence location '''


df8 = df8.reset_index(drop=True)
ind = df8[df8['D_1'] == '1'].index
housestop = df8[df8['D_1'] == '2'].index
x = np.diff(np.searchsorted(housestop,ind))
y = np.bincount(x)
ii = np.nonzero(y)[0]
zip(ii,y[ii])

In [ ]:
'''study type of product'''

product_type = ['1','2','3','4','5','6','7','8','9','10']
product_name = ['live animals','perishable food products',
                     'non-perishable food products','bonded goods(alcohol, tobacco)',
                     'mineral products','chemicals',
                     'sundry items','pharmaceuticals',
                     'rubber and leather','wood,paper']

'''find counts of product'''
def find_product(df, category):
    product = df[df.index.isin([df.index[ind] for ind in range(len(df.index))
                                if (category in df.index[ind])])]['freq'].sum()
    return product

'''convert list into df'''
def get_df_product(df):
    df_product = pd.DataFrame()
    df_product['product_type'] = product_type
    df_product['product_name'] = product_name
    df_product['count'] = [find_product(df, product) for product in product_type]
    df_product['count'].iloc[0] = df_product['count'].iloc[0] - df_product['count'].iloc[9]  # modify due to text identification
    return df_product

def plot_product(df, word):
    df_product = get_df_product(df)
    fig, ax = plt.subplots()
    plt.barh(y = np.arange(len(df_product)), width = df_product['count'])
    ax.set(yticks = np.arange(len(df_product)), yticklabels = df_product.product_name)
    plt.xlabel('number of times delivered')
    plt.title('type of products delivered by %s' %(word))
    #plt.title('type of products delivered by '{}'.format(word)')
    

#df27 = pd.DataFrame(van.groupby('D_4').size().rename('freq'))
#df_product_van = get_df_product(df27)
#df_product_van


In [ ]:
'''should be resolved now (wierd result)'''
'''dangerous to use df345?'''

fig, ax = plt.subplots(figsize = (10,8))

df27 = pd.DataFrame(van.groupby('D_4').size().rename('freq'))
df28 = pd.DataFrame(truck.groupby('D_4').size().rename('freq'))
df37 = df345[df345['D_1'].isin(['2','5'])]
df38 = pd.DataFrame(df37.groupby('D_4').size().rename('freq'))
df35 = df37[df37['D_2'] == '8.0']
df29 = pd.DataFrame(df35.groupby('D_4').size().rename('freq'))

df31 = get_df_product(df27)  # van
df32 = get_df_product(df28)  # truck
df33 = get_df_product(df29)  # household

'''df39/df38 is all vehicles that have identified stops '''
df39 = get_df_product(df38)  # all

ind39 = np.arange(len(df39))
ind31 = np.arange(len(df31))
ind32 = np.arange(len(df32))
height = 0.4
plt.barh(y = ind39, width = df39['count'], height = 0.55 * height, label = 'all')
plt.barh(y = ind31 + 0.5 * height, width = df31['count'], height = 0.55 * height, label = 'van')
plt.barh(y = ind32 + height, width = df32['count'], height = 0.55 * height, label = 'truck')
ax.set(yticks=ind31 + 0.5 * height, yticklabels=df31.product_name)
plt.legend()
plt.title('type of products delivered for different vehicle type')

In [ ]:
check_order = pd.DataFrame(columns = ['id','check'])
for i,user in enumerate(df66.ID.unique()):
    l = df66[df66.ID == user].DS_2
    if len(l) > 1:
        check = all(l[i] <= l[i+1] for i in xrange(len(l)-1))
        check_order.loc[i] = [user, check]
    elif len(l) == 1: check_order.loc[i] = [user, False]
    else: check_order.loc[i] = [user, False]
    #check_order.append()
check_order

In [ ]:
coords = df6[df6.ID == 'XE3629G'].as_matrix(columns=['lat', 'lon'])
#db = DBSCAN(eps=eps, min_samples=ms, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
kms_per_radian = 6371.0088
epsilon = 0.05 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=10, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True

unique_labels = set(cluster_labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (cluster_labels == k)
    X = np.radians(coords)
    #X = StandardScaler().fit_transform(X)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % num_clusters)
plt.show()

In [ ]:
df66[['diff_distance','time']].groupby(pd.Grouper(key='time', freq="M")).mean().plot()

In [ ]:
{row[0]:row[-1] for ind,row in df64.iterrows()}

In [ ]:
#import json, requests
#url = 'https://api.foursquare.com/v2/venues/VENUE_ID/listed'
url = 'https://api.foursquare.com/v2/venues/search'

params = dict(
  client_id='',
  client_secret='',
  v='20190814',
  #ll='1.3413,103.9638',
  #ll='1.3514986,103.9401629',  
  ll='1.282302, 103.858528',  
  radius=2000,  
  #VENUE_ID='HZXXY3Y',  
  limit=2000
  #query='coffee',
  #limit=1
)
resp = requests.get(url=url, params=params)
data1 = json.loads(resp.text)

len(data1['response']['venues'])

In [ ]:
list_id = []
name = []
lat = []
lng = []
postcode = []
address = []
category = []
for query in data1['response']['venues']:
    list_id.append(query['id'])
    name.append(query['name'])
    lat.append(query['location']['lat'])
    lng.append(query['location']['lng'])
    #postcode.append(query['venue']['location']['postalCode'])
    #postcode.append(query['venue']['location']['formattedAddress'][1])
    #if query['location']['postalCode']:
        #postcode.append(query['location']['postalCode'])
    #else: postcode.append('')
    address.append(query['location']['formattedAddress'])
    if query['categories']: 
        category.append(query['categories'][0]['name'])
    else: category.append('')
    #category.append(query['categories'][0]['name'] if (len(query['categories'] > 0) else: ''))

poi_data1 = pd.DataFrame({'id': list_id, 
                         'name': name,
                         'lat': lat,
                         'lng': lng,
                         #'postcode': postcode,
                         'address': address,
                         'category': category})